In [48]:
import matplotlib.pyplot as plt
import time
from IPython import display
from xml.dom import minidom
import math
from exif import Image
import base64
import folium
import os
from datetime import datetime
import random


In [49]:
colors = ["black","silver","gray","maroon","red","purple","fuchsia","green","lime","olive","yellow","navy","blue","teal","aqua","aliceblue","antiquewhite","aqua","aquamarine","beige","bisque","black","blanchedalmond","blue","blueviolet","brown","burlywood","cadetblue","chartreuse","chocolate","coral","cornflowerblue","crimson","cyan","darkblue","darkcyan","darkgoldenrod","darkgray","darkgreen","darkgrey","darkkhaki","darkmagenta","darkolivegreen","darkorange","darkorchid","darkred","darksalmon","darkseagreen","darkslateblue","darkslategray","darkslategrey","darkturquoise","darkviolet","deeppink","deepskyblue","dimgray","dimgrey","dodgerblue","firebrick","forestgreen","fuchsia","gainsboro","gold","goldenrod","gray","green","greenyellow","grey","honeydew","hotpink","indianred","indigo","khaki","lavenderblush","lawngreen","lemonchiffon","lightblue","lightcoral","lightgoldenrodyellow","lightgray","lightgreen","lightgrey","lightpink","lightsalmon","lightseagreen","lightskyblue","lightslategray","lightslategrey","lightsteelblue","lime","limegreen","linen","magenta","maroon","mediumaquamarine","mediumblue","mediumorchid","mediumpurple","mediumseagreen","mediumslateblue","mediumspringgreen","mediumturquoise","mediumvioletred","midnightblue","mistyrose","moccasin","navajowhite","navy","oldlace","olive","olivedrab","orange","orangered","orchid","palegoldenrod","palegreen","paleturquoise","palevioletred","papayawhip","peachpuff","peru","pink","plum","powderblue","purple","red","rosybrown","royalblue","saddlebrown","salmon","sandybrown","seagreen","seashell","sienna","silver","skyblue","slateblue","slategray","slategrey","springgreen","steelblue","tan","teal","thistle","tomato","turquoise","violet","wheat","whitesmoke","yellow","yellowgreen"]

In [50]:
gpx_folder = 'C:\\Users\\TN90072\\Documents\\Python_Scripts\\Problems_VScode\\Germany GPX'
gpx_files = []
for root, dirs, files in os.walk(gpx_folder,topdown = True):
    for gpx in files: 
        _, ending = os.path.splitext(gpx)
        if ending == ".gpx":
            gpx_files.append(os.path.join(root,gpx))


dlon,dlat=minidom.parse(gpx_files[0]).getElementsByTagName('trkpt')[0].attributes['lon'].value,minidom.parse(gpx_files[0]).getElementsByTagName('trkpt')[0].attributes['lat'].value
myMap = folium.Map(location=[dlon, dlat], zoom_start=12)

for gpx in gpx_files:
    data=open(gpx)
    xmldoc = minidom.parse(data)
    track = xmldoc.getElementsByTagName('trkpt')
    elevation=xmldoc.getElementsByTagName('ele')
    date_time=xmldoc.getElementsByTagName('time')
    n_track=len(track)
    
    #PARSING GPX ELEMENT
    lon_list=[]
    lat_list=[]
    for s in range(n_track):
        lon,lat=track[s].attributes['lon'].value,track[s].attributes['lat'].value
        lon_list.append(float(lon))
        lat_list.append(float(lat))
    start_finish = [date_time[0].firstChild.nodeValue, date_time[-1].firstChild.nodeValue]
    start_finish = [datetime.strptime(start_finish[0], "%Y-%m-%dT%H:%M:%S%z").timestamp(), datetime.strptime(start_finish[1], "%Y-%m-%dT%H:%M:%S%z").timestamp()]
    points = []
    for i in range(len(lon_list)):
        points.append([lat_list[i], lon_list[i]])

    color = random.choice(colors)
    print(gpx, end=": ")
    print(color)
    folium.PolyLine(points, color=color).add_to(myMap)

C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\1 августа 2019.gpx: indigo
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\11 августа 2019.gpx: bisque
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\12 августа 2019.gpx: mediumaquamarine
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\13 августа 2019 - 2.gpx: navy
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\13 августа 2019.gpx: lavenderblush
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\14 августа 2019.gpx: lightgreen
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\15 августа 2019.gpx: darkorange
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\18 августа 2019.gpx: greenyellow
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\2 августа 2019.gpx: royalblue
C:\Users\TN90072\Documents\Python_Scripts\Problems_VScode\Germany GPX\27 июля 2019.gp

In [51]:
foldername = "C:\\Users\\TN90072\\Documents\\Python_Scripts\\Problems_VScode"
images = []
for root, dirs, files in os.walk(foldername,topdown = True):
    for name in files: 
        _, ending = os.path.splitext(name)
        if ending == ".jpg":
            images.append(os.path.join(root,name))

In [52]:
for image in images:
    with open(image, 'rb') as image_file:
        my_image = Image(image_file)
        
        if my_image.has_exif == False:
            #print("File {} has no EXIF".format(image))
            continue

        try:
            lat_deg = int(my_image.gps_latitude[0])
            lat_min = int(my_image.gps_latitude[1])
            lat_sec = my_image.gps_latitude[2]

            lon_deg = int(my_image.gps_longitude[0])
            lon_min = int(my_image.gps_longitude[1])
            lon_sec = my_image.gps_longitude[2]

            lat_sign = my_image.gps_latitude_ref
            lon_sign = my_image.gps_longitude_ref

            decimal_lat = lat_deg + lat_min/60 + lat_sec/3600
            decimal_lon = lon_deg + lon_min/60 + lon_sec/3600
        except AttributeError:
            #print("File {} has no coordinates".format(image))
            continue

        if lat_sign == "S":
            decimal_lat *= -1

        if lon_sign == "W":
            decimal_lon *= -1

        #photo_time = datetime.strptime(my_image.datetime, "%Y:%m:%d %H:%M:%S").timestamp()
        #if start_finish[1] > photo_time > start_finish[0]:
        #Add Marker
        iframe = '''<iframe width="400" height="225" frameBorder="0" src="{}"</iframe>'''.format('file:///{}'.format(image).replace("\\","/"))
        popup = folium.Popup(iframe, max_width=420)

        folium.Marker(location=[decimal_lat, decimal_lon], tooltip=iframe, popup = popup, 
        icon=folium.Icon(color = 'gray', icon = 'image', prefix='fa')).add_to(myMap) # https://fontawesome.com/icons/image

In [53]:
myMap.save("index2.html")

In [54]:
        """
        photo_time = datetime.strptime(my_image.datetime, "%Y:%m:%d %H:%M:%S").timestamp()
        #if start_finish[1] > photo_time > start_finish[0]:
        #Add Marker
        iframe = '''<iframe width="400" height="225" frameBorder="0" src="{}"</iframe>'''.format('file:///{}'.format(image).replace("\\","/"))
        popup = folium.Popup(iframe, max_width=420)

        folium.Marker(location=[decimal_lat, decimal_lon], tooltip=iframe, popup = popup, 
        icon=folium.Icon(color = 'gray', icon = 'image', prefix='fa')).add_to(myMap) # https://fontawesome.com/icons/image
        """

'\nphoto_time = datetime.strptime(my_image.datetime, "%Y:%m:%d %H:%M:%S").timestamp()\n#if start_finish[1] > photo_time > start_finish[0]:\n#Add Marker\niframe = \'\'\'<iframe width="400" height="225" frameBorder="0" src="{}"</iframe>\'\'\'.format(\'file:///{}\'.format(image).replace("\\","/"))\npopup = folium.Popup(iframe, max_width=420)\n\nfolium.Marker(location=[decimal_lat, decimal_lon], tooltip=iframe, popup = popup, \nicon=folium.Icon(color = \'gray\', icon = \'image\', prefix=\'fa\')).add_to(myMap) # https://fontawesome.com/icons/image\n'